<a href="https://colab.research.google.com/github/funpi89/NLP_demo/blob/master/chinese_seg_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import pickle
import codecs
import glob
import math
import random
import builtins
import string
import numpy as np
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [0]:
def str_full_to_half(in_str):
  out_str = []
  for char in in_str:
      inside_code = ord(char)
      if inside_code == 0x3000 or inside_code == 12288 or char==string.whitespace: # 全形空格直接轉換
            out_str.append(' ')
      elif inside_code >= 65281 and inside_code <= 65374:
          inside_code -= 0xfee0
          out_str.append(chr(inside_code))
      else:
          out_str.append(char)
      
  return ''.join(out_str)

In [0]:
as_train = codecs.open('/content/drive/My Drive/modern_NLP/icwb2-data/training/as_training.utf8',encoding='utf-8-sig').read()
cityu_train = codecs.open('/content/drive/My Drive/modern_NLP/icwb2-data/training/cityu_training.utf8',encoding='utf-8-sig').read()

#兩個數據集的分割符號不太一樣
as_train = as_train.replace('\u3000','|').replace(' ','|')   #把分詞分隔號置換為'|'，否則會被視為空白被處理掉
cityu_train = cityu_train.replace(' ','|')   #把分詞分隔號置換為'|'，否則會被視為空白被處理掉

data = as_train + '\r\n' + cityu_train #把兩個語料合併
data = data.strip() #去除無效的字元
#as_train=as_train.translate(str.maketrans('０１２３４５６７８９', '0123456789')) #把全形數字轉半形(使用translate)
data = str_full_to_half(data) #把所有全形轉半形

raw_data_train = data.split('\r\n')#分行

raw_data_train = [row.strip('\n').strip('\r').replace("\x08",'').replace("\x80",'') for row in raw_data_train] #移除分行字元

print(raw_data_train[:20])

train_data_len = [len(txt.split('|'))  for txt in  raw_data_train]
print(max(train_data_len))

['時間|:', '三月|十日|(|星期四|)|上午|十時|。', '地點|:', '學術|活動|中心|一樓|簡報室|。', '主講|:', '民族所|所長|莊英章|先生|。', '講題|:', '閩|、|台|漢人|社會|研究|的|若干|考察|。', '李|院長|於|二月|二十六日|至|三月|十五日|赴|美|訪問|,', '期間|將|與|在|美|院士|商討|院務|,', '與|美國|大學|聯繫|商討|長期|合作|事宜|,', '並|辦理|加州|大學|退休|等|手續|。', '出國|期間|院務|由|羅|副院長|代行|。', '總辦事處|秘書組|主任|戴政|先生|請辭|獲准|,', '所|遺|職務|自|三月|一日|起|由|近代史|研究所|研究員|陶英惠|先生|兼任|。', '植物|研究所|所長|周昌弘|先生|當選|第三世界|科學院|(|The|Third|World|Academy|of|Sciences|,', '簡稱|TWAS|)|院士|。', 'TWAS|係|一九八三年|由|Prof|Adbus|Salam|(|巴基斯坦籍|,', '曾|獲|諾貝爾獎|)|發起|成立|,', '會員|遍佈|63|個|國家|,']
211


In [0]:
as_test = codecs.open('/content/drive/My Drive/modern_NLP/icwb2-data/gold/as_testing_gold.utf8',encoding='utf-8-sig').read()
cityu_test = codecs.open('/content/drive/My Drive/modern_NLP/icwb2-data/gold/cityu_test_gold.utf8',encoding='utf-8-sig').read()

#兩個數據集的分割符號不太一樣
as_test = as_test.replace('\u3000','|').replace(' ','|')   #把分詞分隔號置換為'|'，否則會被視為空白被處理掉
cityu_test = cityu_test.replace(' ','|')   #把分詞分隔號置換為'|'，否則會被視為空白被處理掉

data_test = as_test+'\r\n'+cityu_test #把兩個語料合併
data_test = data_test.strip() #去除無效的字元
#as_train=as_train.translate(str.maketrans('０１２３４５６７８９', '0123456789')) #把全形數字轉半形(使用translate)
data_test = str_full_to_half(data_test) #把所有全形轉半形

raw_data_test = data_test.split('\r\n')#分行

raw_data_test = [row.strip('\n').strip('\r').replace("\x08",'').replace("\x80",'') for row in raw_data_test] #移除分行字元

print(raw_data_test[:20])
test_data_len = [len(txt.split('|'))  for txt in  raw_data_test]
print(max(test_data_len))

['許多|社區|長青|學苑|多|開設|有|書法|、|插花|、|土風舞班|,', '文山區|長青|學苑|則|有|個|十分|特別|的|「|英文|歌唱班|」|,', '成員|年齡|均|超過|六十|歲|,', '這|群|白髮蒼蒼|,', '爺爺|、|奶奶級|的|學員|唱起|英文|歌|來|字正腔圓|,', '有模有樣|。', '對|他們|來說|,', '能|與|一|群|志同道合|的|朋友|共同|回味|年少|時期|流行|的|歌曲|,', '才|是|參加|英文|歌唱班|最|大|樂趣|。', '長青|學苑|英文|歌唱班|昨日|在|社教館|舉行|「|英文|懷念|金曲|演唱會|」|,', '曲目|包括|「|大江東去|」|、|「|月河|」|、|「|真善美|」|等|大眾|耳熟能詳|的|英文|歌曲|。', '難得|公開|演唱|,', '這些|有|著|豐富|人生|閱歷|的|學員|絲毫|不|覺得|緊張|怯場|,', '只|見|台|上|唱|得|盡興|,', '台|下|不少|聽眾|也|一時|技癢|跟|著|唱和|起來|。', '長青|學苑|英文|歌唱班|成立|至今|已|兩|年|,', '目前|成員|約|廿五|人|,', '年齡|都|在|六十|歲|以上|,', '其中|以|軍公教|退休|人員|居多|,', '並|有|現任|大學|教授|,']
104


In [0]:
vocabs = sorted(set(list(''.join(raw_data_train))))
vocabs.remove('|')
vocabs.insert(0, '/Unknow')
vocabs.insert(0, '<PAD>')

print(len(vocabs))
print(vocabs[:10])

6296
['<PAD>', '/Unknow', ' ', '!', '$', '%', '&', "'", '(', ')']


In [0]:
char_to_idx = {w:i for i,w in enumerate(vocabs)}
idx_to_char = {i:w for i,w in enumerate(vocabs)}

print({key:char_to_idx[key] for key in list(char_to_idx.keys())[:100]})

{'<PAD>': 0, '/Unknow': 1, ' ': 2, '!': 3, '$': 4, '%': 5, '&': 6, "'": 7, '(': 8, ')': 9, '*': 10, '+': 11, ',': 12, '-': 13, '.': 14, '/': 15, '0': 16, '1': 17, '2': 18, '3': 19, '4': 20, '5': 21, '6': 22, '7': 23, '8': 24, '9': 25, ':': 26, ';': 27, '<': 28, '=': 29, '?': 30, '@': 31, 'A': 32, 'B': 33, 'C': 34, 'D': 35, 'E': 36, 'F': 37, 'G': 38, 'H': 39, 'I': 40, 'J': 41, 'K': 42, 'L': 43, 'M': 44, 'N': 45, 'O': 46, 'P': 47, 'Q': 48, 'R': 49, 'S': 50, 'T': 51, 'U': 52, 'V': 53, 'W': 54, 'X': 55, 'Y': 56, 'Z': 57, '\\': 58, '_': 59, '`': 60, 'a': 61, 'b': 62, 'c': 63, 'd': 64, 'e': 65, 'f': 66, 'g': 67, 'h': 68, 'i': 69, 'j': 70, 'k': 71, 'l': 72, 'm': 73, 'n': 74, 'o': 75, 'p': 76, 'q': 77, 'r': 78, 's': 79, 't': 80, 'u': 81, 'v': 82, 'w': 83, 'x': 84, 'y': 85, 'z': 86, '{': 87, '}': 88, '°': 89, '·': 90, '×': 91, 'ˊ': 92, 'ˋ': 93, '˙': 94, 'Α': 95, 'Β': 96, 'Γ': 97, 'Ε': 98, 'Η': 99}


In [0]:
idx_train = 0
idxs_train = np.arange(len(raw_data_train))
np.random.shuffle(idxs_train)

idx_test = 0
idxs_test = np.arange(len(raw_data_test))
np.random.shuffle(idxs_test)

def get_next_minibatch(minibatch_size, is_Train=True):
  global idx_train, idxs_train, raw_data_train, idx_test, idxs_test, raw_data_test

  groundtruths = []
  idx = idx_train
  idxs = idxs_train
  raw_data = raw_data_train
  if is_Train == False:
    idx = idx_test
    idxs = idxs_test
    raw_data = raw_data_test

  np.random.shuffle(raw_data)

  feature_arr = np.zeros((minibatch_size, 128))
  label_arr = np.zeros((minibatch_size, 128))
  length_arr = np.zeros((minibatch_size))
  batch = 0

  while batch < minibatch_size:
    seq_len = len(raw_data[idxs[idx]].replace('|',''))
    if seq_len <= 128 and seq_len > 0:
      length_arr[batch]=seq_len

      groundtruth_seq = []
      words = raw_data[idxs[idx]].split('|')

      pos=0
      #BMES=>[0,1,2,3]
      for word in words:
        for i in range(len(word)):
          if word[i] in char_to_idx:
            feature_arr[batch, pos] = char_to_idx[word[i]]
          else:
            feature_arr[batch, pos] = char_to_idx['/Unknow']

          if len(word) == 1 and i == 0: #S
            label_arr[batch, pos] = 3
          elif i == 0: #B
            label_arr[batch, pos] = 0
          elif i == len(word)-1: #E
            label_arr[batch, pos] = 2
          else: #M
            label_arr[batch, pos] = 1  
          pos += 1

        groundtruth_seq.append(word)
      groundtruths.append(groundtruth_seq)
      batch += 1
    
    idx += 1
    
    if idx > len(idxs):
      idx = 0
      np.random.shuffle(idxs)

  idx_train = idx
  if is_Train == False:
    idx_test = idx
  return feature_arr.astype(np.int64), label_arr.astype(np.int64), length_arr.astype(np.int64), groundtruths



In [0]:
x_feature,y_label,x_len,ground_truths=get_next_minibatch(2)
print(x_feature.shape)
print(y_label.shape)
print(x_len)
print(ground_truths)

(2, 128)
(2, 128)
[4 4]
[['鈉', '滯留', ','], ['沒有', '錢', ',']]


In [0]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
from torch.autograd import Variable

In [0]:
class CharRNN(nn.Module):
  def __init__(self, batch_size, vocab_size, embed_size, hidden_size, output_size):
    super(CharRNN, self).__init__()
    self.batch_size = batch_size
    self.vocab_size = vocab_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.embed_size = embed_size
    self.embedding = nn.Embedding(vocab_size, embed_size,padding_idx=0)
    self.lstm = nn.LSTM(input_size=self.embed_size, hidden_size=self.hidden_size,
                        num_layers=2,
                        batch_first=True,
                        bidirectional=True)
    self.dropout = nn.Dropout(0.5)
    self.linear = nn.Linear(hidden_size*2, output_size)

  def init_hidden(self):
    hidden_a = torch.randn(2*2, self.batch_size, self.hidden_size).to(device)
    hidden_b = torch.randn(2*2, self.batch_size, self.hidden_size).to(device)
    hidden_a = Variable(hidden_a)
    hidden_b = Variable(hidden_b)
    return (hidden_a, hidden_b)

  def forward(self, x, x_lengths):
    self.hidden = self.init_hidden()

    output = self.embedding(x) # batch, seq_len, embed_dim
    output = torch.nn.utils.rnn.pack_padded_sequence(output, x_lengths, batch_first=True)

    output, self.hidden = self.lstm(output, self.hidden)
    output, _ = torch.nn.utils.rnn.pad_packed_sequence(output, batch_first=True) # batch, seq_len, hidden_size*2

    output = output.contiguous()
    output = output.view(-1, output.shape[2]) # batch*seq_len, hidden_size

    output = self.dropout(output)
    output = self.linear(output) # batch*seq_len, output_size(BMES)

    output = F.softmax(output, dim=1)
    output = output.view(self.batch_size, output.size(0)//self.batch_size, self.output_size)
    return output

In [0]:
def masked_loss(pred_y, target_y, input_x, num_class=5):
  pred_y = pred_y.view(-1, num_class)
  target_y = target_y.view(-1)[:pred_y.shape[0]]
  input_x = input_x.view(-1)[:pred_y.shape[0]]

  mask = (input_x > 0).float()

  nb_tokens = int(torch.sum(mask).data.item())

  pred_y = (1 - pred_y[range(pred_y.shape[0]), target_y]) * mask

  ce_loss = torch.sum(pred_y) / nb_tokens
  return ce_loss

In [0]:
learning_rate=0.0001
minibatch_size=64
num_epochs=20

#宣告模型結構
#output_size還是4，不要計算pad
model = CharRNN(batch_size=minibatch_size,vocab_size=len(vocabs),embed_size=256, hidden_size=256, output_size=4)
model.to(device)
if not os.path.exists('Models'):
  os.mkdir('Models')
  print("Directory Models Created ")
  
if os.path.exists('Models/word_segment_pytorch.lstm'):
  model=torch.load('Models/word_segment_pytorch.lstm')
  print('recovered!!')

model.train()
model_optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.95, 0.999))
criterion = nn.NLLLoss()



epochs=30
accuracy_numerator=[]
accuracy_denominator=[]
print('start epoch!')
for epoch in range(epochs):
  mbs=0
  while mbs<2000:
    features, labels,lens,ground_truths=get_next_minibatch(minibatch_size)
    inputs, targets,seq_len = torch.from_numpy(features.astype(np.int64)), torch.from_numpy(labels.astype(np.int64)), torch.from_numpy(lens.astype(np.int64))

    inputs, targets,seq_len = Variable(inputs), Variable(targets), Variable(seq_len).to(device)
    
    #所有輸入的sequence必須由長到短排序才可以輸入
    seq_len_sorted, sorted_idx = seq_len.sort(descending=True)
    inputs_sorted, targets_sorted,seq_len_sorted=inputs[sorted_idx].to(device), targets[sorted_idx].to(device),seq_len_sorted.to(device)
    
    model_optimizer.zero_grad()
    
      
    output= model.forward(inputs_sorted,seq_len_sorted)
    loss= masked_loss(output,targets_sorted,inputs_sorted,4)
    loss.backward()
    
    output_arr=np.argmax(output.data.cpu().numpy(),-1)
    target_arr=targets_sorted.data.cpu().numpy()[:,:output_arr.shape[1]]
    mask=np.greater(inputs_sorted.data.cpu().numpy(),0)[:,:output_arr.shape[1]]
    
    accuracy_numerator.append((np.equal(output_arr,target_arr)*mask).sum())
    accuracy_denominator.append(mask.sum())
    

    model_optimizer.step()
    if mbs % 100 == 0:
        #列印訓練狀態
        print("Epoch: {}/{}...".format(epoch+1, epochs),
                  "Step: {}...".format(mbs),
                  "Loss: {:.4f}...".format(loss.data.item()),
                  "Accuracy: {:.3%}...".format(sum(accuracy_numerator)/sum(accuracy_denominator)))
        torch.save(model, 'Models/word_segment_pytorch.lstm')
        accuracy_numerator=[]
        accuracy_denominator=[]
    if mbs % 500 == 0:
        learning_rate*=0.75
        test_features, test_labels,test_lens,test_ground_truths=get_next_minibatch(minibatch_size,False)
        inputs, targets,seq_len = torch.from_numpy(test_features.astype(np.int64)), torch.from_numpy(test_labels.astype(np.int64)), torch.from_numpy(test_lens.astype(np.int64))
        inputs, targets,seq_len = Variable(inputs), Variable(targets), Variable(seq_len).to(device)
        #所有輸入的sequence必須由長到短排序才可以輸入
        seq_len_sorted, sorted_idx = seq_len.sort(descending=True)
        inputs_sorted, targets_sorted,seq_len_sorted=inputs[sorted_idx].to(device), targets[sorted_idx].to(device),seq_len_sorted.to(device)
        output= model.forward(inputs_sorted,seq_len_sorted)
        sorted_idx_arr=sorted_idx.data.cpu().numpy()
        result=np.argmax(output.data.cpu().numpy(),-1)
        print('-----測試集驗證--------')
        for i in range(3):
          answer='|'.join(test_ground_truths[sorted_idx_arr[i]])+'|'
          pred=[]
          words=list(''.join(test_ground_truths[sorted_idx_arr[i]]))
          for j in range(len(words)):
            word=words[j]
            onehot=result[i][j]
            if onehot>=2:
              pred.append(word+'|')
            else:
              pred.append(word)

          pred=''.join(pred)  
          print('predict:'+pred)
          print('answer :'+answer)
        print('-----------------')
        
    mbs += 1

start epoch!
Epoch: 1/30... Step: 0... Loss: 0.7506... Accuracy: 27.866%...
-----測試集驗證--------
predict:博達科技疑遭|掏空案,|正|由士|林地檢署指揮調查局如火如荼偵查中,由於|博達董|事長|葉素菲對六十三億|元|應收帳款交代|不清,有檢調人員懷疑,六十三億|資金|中,有|一部分可能是|不|實|在|的|「|虛|擬|交易」,有部分則可能輾|轉匯回國|內替|博達股票|護盤,|拉抬股價,|吸引投|資|人|資|金。|
answer :博達|科技|疑|遭|掏空|案|,|正|由|士林|地檢署|指揮|調查局|如火如荼|偵查|中|,|由於|博達|董事長|葉素菲|對|六十三億|元|應收|帳款|交代|不|清|,|有|檢調|人員|懷疑|,|六十三億|資金|中|,|有|一|部分|可能|是|不|實在|的|「|虛擬|交易|」|,|有|部分|則|可能|輾轉|匯|回|國內|替|博達|股票|護盤|,|拉抬|股價|,|吸引|投資人|資金|。|
predict:這|部戲籌|備13個月,拍|了11個月,|不|得|不|草率殺青,米高梅|大|賠其錢,|有人認為馬龍|白|蘭|度|的「|叛|艦喋|血記」與|伊麗莎白泰|勒的「|埃|及艷后」,宣判好萊塢明星制度死亡|。
answer :這|部|戲|籌備|13|個|月|,|拍|了|11|個|月|,|不得不|草率|殺青|,|米高梅|大|賠|其|錢|,|有人|認為|馬龍白蘭度|的|「|叛艦|喋血記|」|與|伊麗莎白泰勒|的|「|埃及|艷后|」|,|宣判|好萊塢|明星|制度|死亡|。|
predict:他說,通|過在|法|國|各|地|開展|的各式|各樣的文化活動,法國人民進一|步|了解了|中國,了解了豐富多彩|的|中國文化,了解了中國人民豐富的創造力和想像力。
answer :他|說|,|通過|在|法國|各|地|開展|的|各式各樣|的|文化|活動|,|法國|人民|進一步|了解|了|中國|,|了解|了|豐富多彩|的|中國|文化|,|了解|了|中國|人民|豐富|的|創造力|和|想像力|。|
-----------------


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type CharRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch: 1/30... Step: 100... Loss: 0.7289... Accuracy: 33.857%...
Epoch: 1/30... Step: 200... Loss: 0.7354... Accuracy: 36.085%...
Epoch: 1/30... Step: 300... Loss: 0.7300... Accuracy: 40.314%...
Epoch: 1/30... Step: 400... Loss: 0.7131... Accuracy: 45.916%...
Epoch: 1/30... Step: 500... Loss: 0.6941... Accuracy: 49.332%...
-----測試集驗證--------
predict:住在東勢鎮受災最|嚴重|的|福隆里東坑路|的|唐家|,|六十九歲|的|阿媽唐范玉金行動不方便|,|與四歲|的|孫子唐沛齊祖孫|住在一樓|,|昨天一夜遭|逢惡水|,|女兒與媳婦在二樓看情|況不對|,|一直高喊|「|媽媽|,|趕快上來|」|,|但雨大風急|,|樓下二人|可|能沒聽到|,|可能雨勢也|太|大|,|祖孫無法推開門|逃|生|。|
answer :住|在|東勢鎮|受災|最|嚴重|的|福隆里|東坑路|的|唐|家|,|六十九|歲|的|阿媽|唐范玉金|行動|不|方便|,|與|四|歲|的|孫子|唐沛齊|祖孫|住|在|一|樓|,|昨天|一|夜|遭逢|惡水|,|女兒|與|媳婦|在|二|樓|看|情況|不對|,|一直|高喊|「|媽媽|,|趕快|上來|」|,|但|雨大風急|,|樓下|二|人|可能|沒|聽到|,|可能|雨勢|也|太|大|,|祖孫|無法|推|開|門|逃生|。|
predict:昨日蔬菜產地溪湖|的|蔬果供應量九十公噸|,|較前(二)日減少百分之二十五|,|因產地受颱風風雨侵襲而無法採收|,|產地價格平均每公斤十五元|,|較前日上漲百分之|五|。|
answer :昨日|蔬菜|產地|溪湖|的|蔬果|供應量|九十|公噸|,|較|前|(|二|)|日|減少|百分之二十五|,|因|產地|受|颱風|風雨|侵襲|而|無法|採收|,|產地|價格|平均|每|公斤|十五|元|,|較|前日|上漲|百分之五|。|
predict:昨天清晨七時許|,|菜公|、|圳寮及西畔等村落|,|突然遭到|「|颶風|